
# Non-Cognitive Interventions — Replication Pipeline (RQ1–RQ4)

This notebook runs the full analysis pipeline and **saves every step** to CSV:
- A *pre-model analysis dataset* for each RQ
- A *results* CSV for each RQ

## Inputs expected
- `data/stud_ex.csv` — student-level summary per experiment
- `data/dat.csv` — problem-level logs (for posttest; item-level)

> If you don’t have `data/dat.csv`, export it from your R preprocessing (`dat`) or build it from the long/problem-level Python frame and write it to `data/dat.csv`.


In [1]:

import os
import numpy as np
import pandas as pd
from pathlib import Path

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools import add_constant
from statsmodels.stats.multitest import multipletests
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM

# Paths
DATA_DIR = Path("data")
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

IN_STUD_EX = DATA_DIR / "stud_ex.csv"
IN_DAT     = DATA_DIR / "dat.csv"

def safe_binary(s):
    """Coerce True/False/'TRUE'/'FALSE'/0/1 to float 0/1."""
    return pd.Series(s).replace({True:1, False:0, "TRUE":1, "FALSE":0}).astype(float)

def holm_adjust(pvals: pd.Series) -> pd.Series:
    mask = pvals.notna().values
    adj = np.full_like(pvals.astype(float).values, np.nan, dtype=float)
    if mask.sum() > 0:
        _, p_holm, _, _ = multipletests(pvals[mask].values, alpha=0.05, method="holm")
        adj[mask] = p_holm
    return pd.Series(adj, index=pvals.index)

print("Data dir:", DATA_DIR.resolve())
print("Outputs dir:", OUT_DIR.resolve())


Data dir: /Users/limorki/Documents/Replication_Educ/noncognitivestudy_replication/output/outputs/data
Outputs dir: /Users/limorki/Documents/Replication_Educ/noncognitivestudy_replication/output/outputs/outputs


In [ ]:

# Load inputs
stud_ex = pd.read_csv('PATH /stud_ex__replication_ready.csv')
print("stud_ex shape:", stud_ex.shape)
display(stud_ex.head())

# dat is optional (only needed for RQ4.2). We try to load it; if missing, we warn.
if IN_DAT.exists():
    dat = pd.read_csv(IN_DAT)
    print("dat shape:", dat.shape)
    display(dat.head())
else:
    dat = None
    print("WARNING: data/dat.csv not found. RQ4.2 will be skipped unless provided.")


stud_ex shape: (16511, 17)


,user_id,assignment_id,psa_id,student_class_id,teacher_id,academic_year,experiment,num_hints,avg_first_response_time,skb_problem_count,avg_sb_accuracy,avg_pt_accuracy,posttest_responses,mastery,treatment,control,condition
0,14671,1210679,PSAV89B,1180471,11341,v2_20_21,Confidence Judgments,0,8.222867e+04,3.0,1.000000,NaN,0,True,0,1,control
1,18550,851142,PSAV89B,1929016,18541,v2_20_21,Confidence Judgments,0,1.278900e+04,1.0,1.000000,NaN,0,False,1,0,Confidence Judgments
2,28497,851140,PSAV89B,1929014,18541,v2_20_21,Confidence Judgments,0,3.811193e+08,5.0,0.800000,NaN,0,False,0,1,control
3,38595,306682,PSA2KQB,760877,707926,v2_20_21,Emotion Labeling,0,1.536425e+04,3.0,1.000000,0.0,2,True,0,1,control
4,48886,519237,PSAV89B,1257226,46660,v2_20_21,Confidence Judgments,25,8.283212e+06,11.0,0.363636,NaN,0,False,0,1,control


## RQ1 — Response Time: `log(avg_first_response_time+1) ~ treatment` (per experiment)

In [4]:

rq1_rows = []
for exp, g in stud_ex.groupby("experiment"):
    sub = g[["avg_first_response_time", "treatment"]].dropna()
    # Save pre-model dataset
    ds_path = OUT_DIR / f"rq1_dataset_{exp.replace(' ', '_')}.csv"
    sub.assign(experiment=exp).to_csv(ds_path, index=False)
    if sub.empty or sub["treatment"].nunique() < 2:
        rq1_rows.append(dict(experiment=exp, Estimate=np.nan, StdErr=np.nan, t=np.nan, p=np.nan, N=len(sub)))
        continue
    sub = sub.copy()
    sub["y"] = np.log1p(sub["avg_first_response_time"].astype(float))
    X = add_constant(sub["treatment"])
    res = sm.OLS(sub["y"], X).fit()
    rq1_rows.append(dict(experiment=exp,
                         Estimate=res.params["treatment"],
                         StdErr=res.bse["treatment"],
                         t=res.tvalues["treatment"],
                         p=res.pvalues["treatment"],
                         N=len(sub)))

rq1 = pd.DataFrame(rq1_rows).sort_values("experiment").reset_index(drop=True)
rq1["p_holm"] = holm_adjust(rq1["p"])
rq1["sig_holm"] = rq1["p_holm"] < 0.05
display(rq1.round(4))

# Save results
rq1.to_csv(OUT_DIR / "rq1_response_time_results.csv", index=False)


,experiment,Estimate,StdErr,t,p,N,p_holm,sig_holm
0,Confidence Judgments,-0.0975,0.0599,-1.6297,0.1032,9316,0.2064,False
1,Emotion Labeling,0.0570,0.0413,1.3792,0.1679,7187,0.2064,False


# Load PostTest Data

In [ ]:
# --- RQ4.2 RECONSTRUCTION FROM events_all__clean.csv (item-level) ---

import numpy as np
import pandas as pd
from pathlib import Path
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM
from statsmodels.stats.multitest import multipletests
from scipy.stats import norm

# Paths
DATA_DIR = Path("processed")  # adjust if your CSV is elsewhere
OUT_DIR  = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Load problem-level events (your "dat")
events = pd.read_csv("ADD YOUR PATH processed/events_all__clean.csv")

# 2) Map psa_id -> experiment name and #posttest items (from the R code)
experiments_map = {
    "PSA2KNM": "Embracing Mistakes",
    "PSA2KNP": "Embracing Mistakes",
    "PSA59TQ": "Inspirational Quotes",
    "PSA7GUA": "Inspirational Quotes",
    "PSA9XWV": "Social Comparison",
    "PSA2KQB": "Emotion Labeling",
    "PSAV89B": "Confidence Judgments",
    "PSAWU6Z": "Confidence Judgments",
}
posttest_n = {
    "PSA2KNM": 2, "PSA2KNP": 2,
    "PSA59TQ": 3, "PSA7GUA": 2,
    "PSA9XWV": 3, "PSA2KQB": 2,
    "PSAV89B": 0, "PSAWU6Z": 0,
}

# 3) Basic derived flags (mirrors the R)
#    - posttest indicator
#    - correct from continuous_score
ev = events.copy()
ev["posttest"] = (ev["control_treatments"] == "posttest").astype(int)
ev["correct"]  = (ev["continuous_score"] == 1).astype(int)

# 4) Define raw per-row control/treatment indicators (as in R)
#    control if startswith "control" OR endswith "plain_message"
#    treatment if startswith "treatment" and NOT endswith "plain_message"
def _endswith_safe(s, suffix):
    s = s.fillna("")
    return s.str.endswith(suffix)

def _startswith_safe(s, prefix):
    s = s.fillna("")
    return s.str.startswith(prefix)

ct = ev["control_treatments"].astype(str)
ev["control_row"]   = (_startswith_safe(ct, "control") | _endswith_safe(ct, "plain_message")).astype(int)
ev["treat_row"]     = (_startswith_safe(ct, "treatment") & (~_endswith_safe(ct, "plain_message"))).astype(int)
ev["video_fail"]    = (ct == "video_check_fail").astype(int)

# 5) Attach experiment & #posttest per psa_id
ev["experiment"] = ev["psa_id"].map(experiments_map)
ev["num_posttest_problems"] = ev["psa_id"].map(posttest_n)

# 6) Aggregate to assignment-level to get final treatment/control flags
#    Grouping key matches the R logic: (user_id, psa_id, assignment_id)
grp_keys = ["user_id", "psa_id", "assignment_id"]
agg = (
    ev.groupby(grp_keys, as_index=False)
      .agg(
          treat_any  = ("treat_row", "sum"),
          control_any= ("control_row", "sum"),
          video_fail_any=("video_fail", "sum"),
          posttest_responses=("posttest", "sum"),
          # keep the first experiment label for this assignment
          experiment=("experiment", "first"),
          num_posttest_problems=("num_posttest_problems","first")
      )
)

# Mark assignment-level treatment/control and "never randomized"
agg["treatment_assign"] = (agg["treat_any"] > 0).astype(int)
agg["control_assign"]   = (agg["control_any"] > 0).astype(int)
agg["video_fail_any"]   = (agg["video_fail_any"] > 0).astype(int)
agg["never_randomized"] = ((agg["treatment_assign"] == 0) & (agg["control_assign"] == 0)).astype(int)

# 7) Keep valid assignments (mirror the R filters)
#    - drop video check fails
#    - drop never_randomized
valid = agg.query("video_fail_any == 0 and never_randomized == 0").copy()

# 8) Keep only the FIRST exposure to each psa_id per student
#    We order by assignment_id (string order is usually stable for the IDs)
valid["running_exp_count"] = (
    valid.sort_values("assignment_id")
         .groupby(["user_id", "psa_id"])
         .cumcount() + 1
)
valid = valid.query("running_exp_count == 1").copy()

# 9) Merge back the assignment-level treatment into the item-level events
ev2 = ev.merge(
    valid[grp_keys + ["treatment_assign","experiment","num_posttest_problems","posttest_responses"]],
    on=grp_keys, how="inner"
)



## RQ2 — Hint Usage: `(num_hints>0) ~ treatment` (logistic, per experiment)

In [33]:

rq2_rows = []
for exp, g in stud_ex.groupby("experiment"):
    sub = g[["num_hints", "treatment"]].dropna().copy()
    # Save pre-model dataset
    ds_path = OUT_DIR / f"rq2_dataset_{exp.replace(' ', '_')}.csv"
    sub.assign(experiment=exp).to_csv(ds_path, index=False)
    if sub.empty or sub["treatment"].nunique() < 2:
        rq2_rows.append(dict(experiment=exp, Estimate=np.nan, StdErr=np.nan, z=np.nan,
                             p=np.nan, OR=np.nan, OR_low=np.nan, OR_high=np.nan, N=len(sub)))
        continue
    sub["hint_used"] = (sub["num_hints"].astype(float) > 0).astype(float)
    X = add_constant(sub["treatment"])
    res = sm.Logit(sub["hint_used"], X).fit(disp=False)
    b = res.params["treatment"]; se = res.bse["treatment"]; z = b/se
    from scipy.stats import norm
    p = 2*(1 - norm.cdf(abs(z)))
    ci_l, ci_u = res.conf_int().loc["treatment"].tolist()
    rq2_rows.append(dict(experiment=exp, Estimate=b, StdErr=se, z=z, p=p,
                         OR=np.exp(b), OR_low=np.exp(ci_l), OR_high=np.exp(ci_u), N=len(sub)))

rq2 = pd.DataFrame(rq2_rows).sort_values("experiment").reset_index(drop=True)
rq2["p_holm"] = holm_adjust(rq2["p"])
rq2["sig_holm"] = rq2["p_holm"] < 0.05
display(rq2.round(4))

# Save results
rq2.to_csv(OUT_DIR / "rq2_hint_usage_results.csv", index=False)


,experiment,Estimate,StdErr,z,p,OR,OR_low,OR_high,N,p_holm,sig_holm
0,Confidence Judgments,-0.1323,0.0434,-3.0451,0.0023,0.8761,0.8046,0.9540,9316,0.0047,True
1,Emotion Labeling,-0.0649,0.0571,-1.1367,0.2557,0.9371,0.8378,1.0482,7195,0.2557,False


## RQ3 — Mastery: `mastery ~ treatment` (logistic, per experiment)

In [34]:

rq3_rows = []
for exp, g in stud_ex.groupby("experiment"):
    sub = g[["mastery", "treatment"]].dropna().copy()
    # Save pre-model dataset
    ds_path = OUT_DIR / f"rq3_dataset_{exp.replace(' ', '_')}.csv"
    sub.assign(experiment=exp).to_csv(ds_path, index=False)
    if sub.empty or sub["treatment"].nunique() < 2:
        rq3_rows.append(dict(experiment=exp, Estimate=np.nan, StdErr=np.nan, z=np.nan,
                             p=np.nan, OR=np.nan, OR_low=np.nan, OR_high=np.nan, N=len(sub)))
        continue
    sub["mastery_bin"] = (sub["mastery"].replace({True:1, False:0, "TRUE":1, "FALSE":0}).astype(float))
    X = add_constant(sub["treatment"])
    res = sm.Logit(sub["mastery_bin"], X).fit(disp=False)
    b = res.params["treatment"]; se = res.bse["treatment"]; z = b/se
    from scipy.stats import norm
    p = 2*(1 - norm.cdf(abs(z)))
    ci_l, ci_u = res.conf_int().loc["treatment"].tolist()
    rq3_rows.append(dict(experiment=exp, Estimate=b, StdErr=se, z=z, p=p,
                         OR=np.exp(b), OR_low=np.exp(ci_l), OR_high=np.exp(ci_u), N=len(sub)))

rq3 = pd.DataFrame(rq3_rows).sort_values("experiment").reset_index(drop=True)
rq3["p_holm"] = holm_adjust(rq3["p"])
rq3["sig_holm"] = rq3["p_holm"] < 0.05
display(rq3.round(4))

# Save results
rq3.to_csv(OUT_DIR / "rq3_mastery_results.csv", index=False)


/var/folders/cj/cjl156792_553wk_b44k9xc40000gn/T/ipykernel_77131/2205093918.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sub["mastery_bin"] = (sub["mastery"].replace({True:1, False:0, "TRUE":1, "FALSE":0}).astype(float))
/var/folders/cj/cjl156792_553wk_b44k9xc40000gn/T/ipykernel_77131/2205093918.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sub["mastery_bin"] = (sub["mastery"].replace({True:1, False:0, "TRUE":1, "FALSE":0}).astype(float))


,experiment,Estimate,StdErr,z,p,OR,OR_low,OR_high,N,p_holm,sig_holm
0,Confidence Judgments,-0.3150,0.0449,-7.0199,0.0,0.7298,0.6683,0.7969,9316,0.0,True
1,Emotion Labeling,-1.6698,0.0723,-23.0912,0.0,0.1883,0.1634,0.2170,7195,0.0,True


## RQ4.1 — Efficiency: `-z(skb_problem_count) ~ treatment` (OLS, mastered only, per experiment)

In [35]:

rq41_rows = []
# Prepare mastered-only dataset
df_e = stud_ex.copy()
df_e["mastery_bin"] = (df_e["mastery"].replace({True:1, False:0, "TRUE":1, "FALSE":0}).astype(float))
df_e = df_e[(df_e["mastery_bin"] == 1) & df_e["skb_problem_count"].notna()].copy()
df_e["skb_problem_count"] = df_e["skb_problem_count"].astype(float)
df_e["efficiency"] = -1 * ( (df_e["skb_problem_count"] - df_e["skb_problem_count"].mean()) / df_e["skb_problem_count"].std(ddof=0) )

for exp, g in df_e.groupby("experiment"):
    sub = g[["efficiency", "treatment"]].dropna().copy()
    # Save pre-model dataset
    ds_path = OUT_DIR / f"rq41_dataset_{exp.replace(' ', '_')}.csv"
    sub.assign(experiment=exp).to_csv(ds_path, index=False)
    if sub.empty or sub["treatment"].nunique() < 2:
        rq41_rows.append(dict(experiment=exp, Estimate=np.nan, StdErr=np.nan, t=np.nan, p=np.nan, N=len(sub)))
        continue
    X = add_constant(sub["treatment"])
    res = sm.OLS(sub["efficiency"], X).fit()
    rq41_rows.append(dict(experiment=exp,
                          Estimate=res.params["treatment"],
                          StdErr=res.bse["treatment"],
                          t=res.tvalues["treatment"],
                          p=res.pvalues["treatment"],
                          N=len(sub)))

rq41 = pd.DataFrame(rq41_rows).sort_values("experiment").reset_index(drop=True)
rq41["p_holm"] = holm_adjust(rq41["p"])
rq41["sig_holm"] = rq41["p_holm"] < 0.05
display(rq41.round(4))

# Save results
rq41.to_csv(OUT_DIR / "rq41_efficiency_results.csv", index=False)


/var/folders/cj/cjl156792_553wk_b44k9xc40000gn/T/ipykernel_77131/4087619300.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_e["mastery_bin"] = (df_e["mastery"].replace({True:1, False:0, "TRUE":1, "FALSE":0}).astype(float))


,experiment,Estimate,StdErr,t,p,N,p_holm,sig_holm
0,Confidence Judgments,-0.2666,0.0311,-8.5785,0.0000,6016,0.0000,True
1,Emotion Labeling,0.0341,0.0187,1.8274,0.0677,5683,0.0677,False


## RQ4.2 — Post-test: `correct ~ treatment + (1|user_id) + (1|problem_id)` (mixed logistic, per experiment)

In [41]:
# START from your long/problem-level DataFrame that has columns like:
# ['user_id','assignment_id','psa_id','experiment','control_treatments',
#  'posttest','mastery','problem_id','correct','section_names','first_response_time',
#  'hint_count','continuous_score', 'control','treatment','treatment1','treatment2', ...]
# If it's named something else (e.g., `df_long`), replace `dat_long` below.

import pandas as pd

#dat = read_csv("/Users/limorki/Documents/Replication_Educ/noncognitivestudy_replication/output/processed/events_all__clean.csv")
#stud_ex = read_csv("data/stud_ex__replication_ready.csv")

# Minimal schema needed for RQ4.2
required = ["user_id","problem_id","experiment","posttest","mastery","treatment","correct"]

# Use ev2, which contains the required columns (experiment_x, posttest, treatment_assign, correct)
# Map columns to match required names
out = ev2.rename(columns={
    "experiment_x": "experiment",
    "treatment_assign": "treatment"
}).copy()

missing = [c for c in required if c not in out.columns]
if missing:
    raise ValueError(f"These columns are missing from your long/problem-level frame: {missing}")

# Clean up types a bit
out["posttest"] = out["posttest"].astype(int)  # 1 if post-test row, else 0
out["mastery"]  = out["mastery"].replace({True:1, False:0, "TRUE":1, "FALSE":0}).astype(int)
out["treatment"]= pd.to_numeric(out["treatment"], errors="coerce").fillna(0).astype(int)
out["correct"]  = pd.to_numeric(out["correct"], errors="coerce").astype(int)

# (Optional) keep only columns we actually use downstream
out = out[required]

# Ensure the 'data' directory exists before writing
import os
os.makedirs("data", exist_ok=True)

# Write to disk
out.to_csv("data/dat.csv", index=False)
print("Wrote data/dat.csv with", len(out), "rows and columns:", list(out.columns))


Wrote data/dat.csv with 113955 rows and columns: ['user_id', 'problem_id', 'experiment', 'posttest', 'mastery', 'treatment', 'correct']


/var/folders/cj/cjl156792_553wk_b44k9xc40000gn/T/ipykernel_77131/1570334962.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out["mastery"]  = out["mastery"].replace({True:1, False:0, "TRUE":1, "FALSE":0}).astype(int)


In [43]:

if dat is None:
    print("dat.csv not available — skipping RQ4.2.")
else:
    rq42_rows = []
    # Use the 'out' DataFrame, which has the correct columns
    dat_post = out.copy()
    dat_post["mastery_bin"] = (dat_post["mastery"].replace({True:1, False:0, "TRUE":1, "FALSE":0}).astype(float))
    dat_post = dat_post[(dat_post.get("posttest", 0) == 1) & (dat_post["mastery_bin"] == 1)].copy()

    for exp, g in dat_post.groupby("experiment"):
        sub = g[["correct", "treatment", "user_id", "problem_id"]].dropna().copy()
        # Save pre-model dataset
        ds_path = OUT_DIR / f"rq42_dataset_{exp.replace(' ', '_')}.csv"
        sub.assign(experiment=exp).to_csv(ds_path, index=False)
        if sub.empty or sub["treatment"].nunique() < 2 or sub["correct"].nunique() < 2:
            rq42_rows.append(dict(experiment=exp, Estimate=np.nan, StdErr=np.nan, z=np.nan, p=np.nan, OR=np.nan, N=len(sub)))
            continue
        try:
            model = BinomialBayesMixedGLM.from_formula(
                "correct ~ treatment",
                vc_formula={"user": "0 + C(user_id)", "problem": "0 + C(problem_id)"},
                data=sub
            )
            fit = model.fit_vb()
            b  = float(fit.fe_params["treatment"])
            se = float(fit.fe_sd["treatment"])
            z  = b / se
            from scipy.stats import norm
            p  = 2*(1 - norm.cdf(abs(z)))
            rq42_rows.append(dict(experiment=exp, Estimate=b, StdErr=se, z=z, p=p, OR=np.exp(b), N=len(sub)))
        except Exception as e:
            rq42_rows.append(dict(experiment=exp, Estimate=np.nan, StdErr=np.nan, z=np.nan, p=np.nan, OR=np.nan, N=len(sub)))

    rq42 = pd.DataFrame(rq42_rows).sort_values("experiment").reset_index(drop=True)
    rq42["p_holm"] = holm_adjust(rq42["p"])
    rq42["sig_holm"] = rq42["p_holm"] < 0.05
    display(rq42.round(4))

    # Save results
    rq42.to_csv(OUT_DIR / "rq42_posttest_results.csv", index=False)


,experiment,Estimate,StdErr,z,p,OR,N,p_holm,sig_holm
0,Emotion Labeling,NaN,NaN,NaN,NaN,NaN,11036,NaN,False



### Notes
- Each RQ cell writes two kinds of artifacts:
  1) **`outputs/rq*_dataset_*.csv`** — the pre-model analysis dataset actually used
  2) **`outputs/rq*_..._results.csv`** — the tidy results table

- RQ4.2 requires `data/dat.csv` (item-level logs). Export `dat` from R or construct it in Python before running that cell.
